In [34]:
import pandas as pd
import numpy as np
import copy
from pandas import *
import scipy.optimize as sopt
import networkx as nx
from networkx.drawing.nx_agraph import to_agraph
import matplotlib.pyplot as plt
from itertools import permutations
from scipy.optimize import minimize_scalar, minimize, root
from scipy.optimize import bisect
np.set_printoptions(precision=3)

### Import Data

In [36]:
link_data = pd.read_excel('link_data.xlsx')
OD_matrix = pd.read_excel('O-D_matrix.xlsx')
link_data['cap'] = link_data['cap'] * 1000
#print(link_data)
#print(OD_matrix)

### Functions Defined

In [37]:
def tt_func(df):
    t0 = np.array(df.t0)
    cap = np.array(df.cap)
    x = np.array(df.x)
    return np.multiply(t0,(1+ 0.15*(np.power(np.divide(x,cap),4))))

def convergence(x_n,x_n1):
    return np.sqrt(np.sum(np.power(np.subtract(x_n1,x_n),2))) / np.sum(x_n)

def return_tt(df):
    x_n = np.array(df['x'])
    t0 = np.array(df.t0)
    ca = np.array(df.cap)
    obj =  np.sum(np.multiply(t0,np.add(x_n,0.15*np.divide(np.power(x_n,4),np.power(ca,4)))))
    return obj

def obj_func(alpha,df):
    y = np.array(df.y)
    x_n = np.array(df.x)
    t0 = np.array(df.t0)
    ca = np.array(df.cap)
    x_sub = np.add(x_n,alpha * np.subtract(y,x_n))
    obj = np.sum(np.multiply(t0,np.add(x_sub,(0.15/5)*np.divide(np.power(x_sub,5),np.power(ca,4)))))
    return obj

### Function for Dijkstra

In [38]:
def dijkstra_sp(orig,dest,df):
    G1=nx.DiGraph()
    #list_nodes=[1,2,3,4,5,6,7,8,9,10,
              # 11,12,13,14,15,16,17,18,19,20,21,22,23,24]
    #G1.add_nodes_from(list_nodes)
    G1.add_edges_from([(link_data['O'][i],link_data['D'][i],{'weight': link_data['tt'][i]}) for i in link_data.index])
    #print(G1.edges(data=True))
    path = nx.dijkstra_path(G1, source=orig, target=dest)
    return path

In [39]:
path=dijkstra_sp(1,6,link_data)
print(path)


[1, 2, 6]


### UE 

In [41]:
### start with intitialization
## Initialization
link_data['x'] = 0
link_data['y'] = 0
Z = []
#T=[]
conv_test = 100
loop = 1
cap = link_data['cap']
while conv_test > 1e-6:
    print('Step : ', loop)
    link_data['y'] = 0
    x_n = np.array(link_data['x'])
    link_data['tt'] = tt_func(link_data)
    for i in range (len(OD_matrix)):
        orig=OD_matrix['O'][i]
        dest=OD_matrix['D'][i]
        fl=OD_matrix['Q'][i]
        path=dijkstra_sp(orig,dest,link_data)
        for k in range(len(path)-1):
            d_new = path[k] 
            o_new = path[k+1]
            link_data.loc[(link_data.O==o_new) & (link_data.D==d_new),'y'] +=fl
    if loop!= 1:
        alpha = minimize_scalar(obj_func,args=(link_data), bounds=(0.0, 1.0),method='bounded').x
    else :
        alpha = 1
    print("Alpha : ",alpha)
    print("Z : ",obj_func(alpha,link_data))
    Z.append(obj_func(alpha,link_data))
    y = np.array(link_data['y'])
    #print(x_n,y)
    x_n1 = x_n + alpha * (y-x_n)
    link_data['x'] = x_n1 ## Update x to update t
    #print("T : ",return_tt(link_data))
    #T.append(return_tt(link_data))
    if loop != 1:
        conv_test= convergence(x_n,x_n1)
        print("Conv : ",conv_test)
    loop += 1
link_data['ratio'] = link_data['x'] / link_data['cap']
print(link_data)
        

Step :  1
Alpha :  1
Z :  1166084.8194119025
Step :  2
Alpha :  0.10114836241844098
Z :  1192466.3027538548
Conv :  0.004341813199692053
Step :  3
Alpha :  5.9608609865491405e-06
Z :  1192466.345780796
Conv :  2.447495417607257e-07
    Link   O   D   t0      cap             x        tt      y     ratio
0      1   1   2  3.6   6020.0   4240.319305  3.732924   3710  0.704372
1      2   1   3  2.4   9010.0  11659.680695  3.409600  12190  1.294082
2      3   2   1  3.6  12020.0   4300.012101  3.608844   6330  0.357738
3      4   2   6  3.0  15920.0  11330.319305  3.115454  10800  0.711703
4      5   3   1  2.4  46810.0  11599.987899  2.401358   9570  0.247810
..   ...  ..  ..  ...      ...           ...       ...    ...       ...
71    72  23  22  2.4  10000.0   3178.714652  2.403675   8410  0.317871
72    73  23  24  1.2  10160.0   4380.292494  1.206219   3050  0.431131
73    74  24  13  2.4  11380.0  11058.065487  2.720960  10330  0.971710
74    75  24  21  1.8   9770.0   8931.604653  1.

In [29]:
# for i in range (len(OD_matrix)):
#         orig=OD_matrix['O'][i]
#         dest=OD_matrix['D'][i]
#         fl=OD_matrix['Q'][i]
#         print(orig,dest,fl)